In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm

# Suppress TensorFlow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate
from tensorflow.keras import backend as K

# --- Constants ---
TRAIN_PATH = '/content/drive/MyDrive/Amazon_ML/student_resource/dataset/train.csv'
TEST_PATH = '/content/drive/MyDrive/Amazon_ML/student_resource/dataset/test.csv'
MAX_NUM_WORDS = 30000
MAX_SEQ_LEN = 150
EMBEDDING_DIM = 128
IMAGE_FEATURE_DIM = 2048 # Dimension expected from a pre-trained ResNet50/EfficientNet
EPSILON = 1e-6 # Small constant for log safety and SMAPE stability

# --- Utility Functions ---

def clean_text(text):
    """Performs standard NLP cleaning."""
    if not isinstance(text, str):
        return ""
    text = text.lower().strip()
    text = re.sub(r"<.*?>", " ", text) # Remove HTML tags
    text = re.sub(r"https?://\S+|www\.\S+", " ", text) # Remove URLs
    text = re.sub(r"[^a-z0-9\s\+\-x%./]", " ", text) # Keep useful symbols
    text = re.sub(r"\s+", " ", text).strip()
    return text

def extract_ipq(text):
    """Extracts Item Pack Quantity (IPQ) using regex."""
    if not isinstance(text, str):
        return 1

    # Aggressive pattern searching for pack quantity
    patterns = [
        r'(\d+)\s*pack', r'pack of\s*(\d+)', r'(\d+)\s*ct\b',
        r'(\d+)\s*count\b', r'(\d+)\s*pcs\b', r'x\s*(\d+)\b'
    ]

    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            try:
                # Return the largest captured number if multiple are found (common for specs)
                return max(1, int(match.group(1)))
            except ValueError:
                continue

    return 1 # Default assumption for single item

def smape_metric(y_true, y_pred):
    """Calculates SMAPE (Symmetric Mean Absolute Percentage Error) - Non-differentiable."""
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_pred) + np.abs(y_true)) / 2.0
    return np.mean(numerator / np.maximum(denominator, EPSILON)) * 100

def custom_smape_loss(y_true, y_pred):
    """Keras custom loss function for SMAPE, operating on exponential values."""

    # 1. Inverse Transform the log-prices back to raw prices
    # y_true and y_pred are log-transformed, we need to exponentiate them
    y_true_raw = K.exp(y_true) - EPSILON
    y_pred_raw = K.exp(y_pred) - EPSILON

    # 2. Apply SMAPE formula
    numerator = K.abs(y_pred_raw - y_true_raw)
    denominator = (K.abs(y_true_raw) + K.abs(y_pred_raw)) / 2.0

    # K.maximum(denominator, K.epsilon()) handles potential division by zero
    return K.mean(numerator / K.maximum(denominator, K.epsilon()), axis=-1)

def download_and_get_image_features(df, feature_dim=IMAGE_FEATURE_DIM):
    """
    CONCEPTUAL FUNCTION: In a real submission, this function must:
    1. Use src/utils.py to download images from image_link.
    2. Load a pre-trained CNN (e.g., ResNet50) without its top layers.
    3. Process each image and extract the feature vector from the penultimate layer.

    Here, we MOCK the process with random data for pipeline completeness.
    """
    print("--- MOCKING IMAGE FEATURE EXTRACTION (Crucial Step) ---")
    # For robust results, replace this with actual CNN feature extraction.
    np.random.seed(42)
    return np.random.rand(df.shape[0], feature_dim).astype(np.float32)

def build_multi_modal_model(input_dim_ipq, input_dim_img, max_words, max_seq_len, embedding_dim):
    """Builds the Keras Functional API model combining Text, IPQ, and Image features."""

    # 1. Text Branch (Convolutional Neural Network for Text)
    text_input = Input(shape=(max_seq_len,), name='text_input')
    text_emb = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_seq_len)(text_input)
    # Using Conv1D and GlobalMaxPooling is often faster and effective than LSTM for text features
    text_conv = Conv1D(filters=128, kernel_size=5, activation='relu')(text_emb)
    text_pool = GlobalMaxPooling1D()(text_conv)
    text_drop = Dropout(0.3)(text_pool)
    text_dense = Dense(64, activation='relu')(text_drop)

    # 2. IPQ (Structural) Branch
    ipq_input = Input(shape=(input_dim_ipq,), name='ipq_input')
    ipq_dense = Dense(32, activation='relu')(ipq_input)
    ipq_drop = Dropout(0.1)(ipq_dense)

    # 3. Image (Visual) Branch
    img_input = Input(shape=(input_dim_img,), name='img_input')
    img_dense = Dense(128, activation='relu')(img_input)
    img_drop = Dropout(0.3)(img_dense)

    # 4. Combine Branches
    combined = Concatenate()([text_dense, ipq_drop, img_drop])

    # 5. Final Regression Head
    dense_final = Dense(64, activation='relu')(combined)
    dense_final = Dropout(0.2)(dense_final)
    output = Dense(1, activation='linear')(dense_final) # Predicts log_price

    model = Model(inputs=[text_input, ipq_input, img_input], outputs=output)

    # Compile using the custom SMAPE loss and a focused optimizer
    model.compile(optimizer='adam', loss=custom_smape_loss)

    return model

# --- Main Execution ---

def main():

    # Load Data
    try:
        train = pd.read_csv(TRAIN_PATH)
        test = pd.read_csv(TEST_PATH)
    except Exception as e:
        print(f"Error loading data. Ensure paths are correct and files exist. Error: {e}")
        return

    # --- 1. Feature Engineering ---

    # Target Transformation: Log(price) for better model convergence
    train['log_price'] = np.log(train['price'].values + EPSILON)
    y_train_log = train['log_price'].values

    # Text Cleaning and IPQ Extraction
    print("1. Cleaning Text and Extracting IPQ...")
    for df in [train, test]:
        df['catalog_content'] = df['catalog_content'].fillna("")
        df['clean_text'] = df['catalog_content'].apply(clean_text)
        df['ipq'] = df['catalog_content'].apply(extract_ipq)

    # IPQ Scaling (Structural Feature)
    ipq_scaler = StandardScaler()
    X_train_ipq = ipq_scaler.fit_transform(train[['ipq']])
    X_test_ipq = ipq_scaler.transform(test[['ipq']])

    # Image Feature Extraction (MOCK)
    print("2. Generating MOCK Image Features (Must be replaced with actual CNN extraction)...")
    X_train_img = download_and_get_image_features(train)
    X_test_img = download_and_get_image_features(test)

    # --- 2. NLP Processing ---

    print("3. Tokenizing and Padding Text...")
    tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(train['clean_text'])

    X_train_seq = tokenizer.texts_to_sequences(train['clean_text'])
    X_test_seq = tokenizer.texts_to_sequences(test['clean_text'])

    X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')
    X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')

    # --- 3. Train/Validation Split ---

    X_tr_text, X_val_text, y_tr, y_val = train_test_split(X_train_pad, y_train_log, test_size=0.1, random_state=42)
    X_tr_ipq, X_val_ipq, _, _ = train_test_split(X_train_ipq, y_train_log, test_size=0.1, random_state=42)
    X_tr_img, X_val_img, _, _ = train_test_split(X_train_img, y_train_log, test_size=0.1, random_state=42)

    # --- 4. Model Building and Training ---

    print("4. Building and Training Multi-Modal Model with Custom SMAPE Loss...")
    model = build_multi_modal_model(X_train_ipq.shape[1], IMAGE_FEATURE_DIM, MAX_NUM_WORDS, MAX_SEQ_LEN, EMBEDDING_DIM)

    history = model.fit(
        {'text_input': X_tr_text, 'ipq_input': X_tr_ipq, 'img_input': X_tr_img}, y_tr,
        validation_data=({'text_input': X_val_text, 'ipq_input': X_val_ipq, 'img_input': X_val_img}, y_val),
        epochs=15, # Increased epochs for better convergence
        batch_size=128,
        verbose=1
    )

    # --- 5. Validation Check ---

    # Predict on validation set (log price)
    y_val_pred_log = model.predict({'text_input': X_val_text, 'ipq_input': X_val_ipq, 'img_input': X_val_img}).flatten()

    # Inverse transform to raw price
    y_val_true_raw = np.exp(y_val) - EPSILON
    y_val_pred_raw = np.exp(y_val_pred_log) - EPSILON
    y_val_pred_raw = np.maximum(y_val_pred_raw, 0.01) # Ensure positive

    # Calculate SMAPE
    val_smape = smape_metric(y_val_true_raw, y_val_pred_raw)
    print(f"\n✅ Validation SMAPE (on {len(y_val)} samples): {val_smape:.4f}%")

    # --- 6. Final Prediction and Submission ---

    print("5. Generating Final Test Predictions...")
    # Predict on the actual test set (log price)
    test_preds_log = model.predict({'text_input': X_test_pad, 'ipq_input': X_test_ipq, 'img_input': X_test_img}).flatten()

    # Inverse Transform
    test_preds = np.exp(test_preds_log) - EPSILON

    # Constraint: Predicted prices must be positive float values.
    test_preds = np.maximum(test_preds, 0.01)

    submission = pd.DataFrame({
        "sample_id": test['sample_id'],
        "price": test_preds
    })

    submission.to_csv("test_out.csv", index=False)
    print("\n📦 Submission file 'test_out.csv' created successfully!")

if __name__ == '__main__':
    # Increase precision for printing
    np.set_printoptions(precision=4)

    # Ensure all TensorFlow operations are run
    # K.clear_session()

    main()

1. Cleaning Text and Extracting IPQ...
2. Generating MOCK Image Features (Must be replaced with actual CNN extraction)...
--- MOCKING IMAGE FEATURE EXTRACTION (Crucial Step) ---
--- MOCKING IMAGE FEATURE EXTRACTION (Crucial Step) ---
3. Tokenizing and Padding Text...
4. Building and Training Multi-Modal Model with Custom SMAPE Loss...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/15
528/528 ━━━━━━━━━━━━━━━━━━━━ 141s 262ms/step - loss: 0.7385 - val_loss: 0.5680
Epoch 2/15
528/528 ━━━━━━━━━━━━━━━━━━━━ 131s 249ms/step - loss: 0.5498 - val_loss: 0.5392
Epoch 3/15
528/528 ━━━━━━━━━━━━━━━━━━━━ 145s 254ms/step - loss: 0.5050 - val_loss: 0.5110
Epoch 4/15
528/528 ━━━━━━━━━━━━━━━━━━━━ 141s 252ms/step - loss: 0.4726 - val_loss: 0.5141
Epoch 5/15
528/528 ━━━━━━━━━━━━━━━━━━━━ 143s 253ms/step - loss: 0.4449 - val_loss: 0.4983
Epoch 6/15
528/528 ━━━━━━━━━━━━━━━━━━━━ 142s 253ms/step - loss: 0.4268 - val_loss: 0.5005
Epoch 7/15
528/528 ━━━━━━━━━━━━━━━━━━━━ 134s 254ms/step - loss: 0.4102 - val_loss: 0.5090
Epoch 8/15
528/528 ━━━━━━━━━━━━━━━━━━━━ 132s 249ms/step - loss: 0.3909 - val_loss: 0.5030
Epoch 9/15
528/528 ━━━━━━━━━━━━━━━━━━━━ 133s 251ms/step - loss: 0.3753 - val_loss: 0.5101
Epoch 10/15
528/528 ━━━━━━━━━━━━━━━━━━━━ 143s 254ms/step - loss: 0.3646 - val_loss: 0.5020
Epoch 11/15
528/528 ━━━━━━━━━━━━━━━━━━━━ 133s 252ms/step - loss: 0.3510 - val_loss: 0.4999
Epoch 12

In [4]:
import numpy as np

# --- Constant required for the SMAPE function ---
EPSILON = 1e-6

def smape_metric(y_true, y_pred):
    """Calculates SMAPE (Symmetric Mean Absolute Percentage Error)."""

    # Ensure y_pred is positive, as is done in your submission logic
    y_pred = np.maximum(y_pred, EPSILON)

    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_pred) + np.abs(y_true)) / 2.0

    # Use np.maximum to prevent division by zero/near zero
    return np.mean(numerator / np.maximum(denominator, EPSILON)) * 100

# --- Example Usage (Mirroring the Validation Check in your code) ---

# Example Actual Raw Prices (y_val_true_raw in your script)
y_actual_raw = np.array([100.0, 250.0, 99.5, 400.0])

# Example Predicted Raw Prices (y_val_pred_raw in your script)
# A very low SMAPE means predictions are very close to actuals
y_predicted_raw = np.array([102.5, 248.0, 99.0, 405.0])

calculated_smape = smape_metric(y_actual_raw, y_predicted_raw)

print(f"Example Actual Prices: {y_actual_raw}")
print(f"Example Predicted Prices: {y_predicted_raw}")
print("-" * 30)
print(f"Calculated SMAPE Score: {calculated_smape:.4f}%")

Example Actual Prices: [100.  250.   99.5 400. ]
Example Predicted Prices: [102.5 248.   99.  405. ]
------------------------------
Calculated SMAPE Score: 1.2546%
